### Open Flights Data Wrangling

To practice, you are going to wrangle data from OpenFlights.  You can read about it here: 

http://openflights.org/data.html

This includes five files: 

1. A file with one record for each airport, 
2. A file with one record for each airline, 
3. A file with one record for each each route, 
4. A file with one record for each type of airplane, and
5. A file with one record for each country.  

Each file has a relationship to the others, with some fields in common.  To make best use of the data, we will need to combine all three files.  This is the core challenge of relational databases.  This lesson looks at how to do that, specifically using the merge() function in python.  

Credits:

This lesson draws from the Data Analysis and Visualization in Python for Ecologists course by Data Carpentry:

https://datacarpentry.org/python-ecology-lesson/05-merging-data/index.html

and from the pandas documentation:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html


### Learning Objectives: 

1. Understand the meaning of relational data and be able to identify common keys. 

2. Understand the different types of relational joins: concatenate, inner join, outer join, left join, right join. 

3. Use the pandas syntax pd.concat() and pd.merge() to perform those relational joins. 

4. Start to build an awareness of open data resources, and of documentation/tutorials for ongoing learning. 

In [2]:
import pandas as pd
import numpy as np

In [3]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["ID", "NAME", "CITY", "COUNTRY", "IATA", "ICAO", "LATITUDE", "LONGITUDE", "ALTITUDE", "TIMEZONE", "DST", "TZ", "TYPE", "SOURCE"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["ID", "NAME", "ALIAS", "IATA", "ICAO", "CALLSIGN", "COUNTRY", "ACTIVE"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["AIRLINE", "AIRLINE_ID", "SOURCE", "SOURCE_ID", "DEST", "DEST_ID", "CODESHARE", "STOPS", "EQUIPMENT"]

# Read in the planes data.
planes = pd.read_csv("data/planes.dat", header=None, na_values='\\N')
planes.columns = ["NAME", "IATA", "ICAO"]

# Read in the countries data.
countries = pd.read_csv("data/countries.dat", header=None, na_values='\\N')
countries.columns = ["NAME", "ISO", "DAFIF"]

### Start by seeing what's in the data.  

What columns are there?  What data types are the columns?  

Remember, 'object' means it is a string, while the numerical values can be floats or ints.  Sometimes you will have problems if it reads numeric data in as strings.  If that happens, you can use the function .astype() to convert it.  Look it up in the pandas API to get more details

Start with the airports table

In [4]:
# printing the first few rows gives us a good sense of what is in there
airports.head()

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports


In [4]:
# we can also look a the data types
# alternatively, we could summarize the data as we did in the previous lesson
airports.dtypes

ID             int64
NAME          object
CITY          object
COUNTRY       object
IATA          object
ICAO          object
LATITUDE     float64
LONGITUDE    float64
ALTITUDE       int64
TIMEZONE     float64
DST           object
TZ            object
TYPE          object
SOURCE        object
dtype: object

##### Your turn:  

Spend a few minutes examining the remaining tables.  See if you can identify which fields are common across multiple tables.  These are the "keys".  Together, we will make sure we have identified the right keys. 

In [5]:
airlines.head()

,ID,NAME,ALIAS,IATA,ICAO,CALLSIGN,COUNTRY,ACTIVE
0,-1,Unknown,NaN,-,NaN,NaN,NaN,Y
1,1,Private flight,NaN,-,NaN,NaN,NaN,Y
2,2,135 Airways,NaN,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,NaN,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,NaN,NaN,WYT,NaN,United Kingdom,N


In [6]:
print(airlines)

         ID                                          NAME              ALIAS  \
0        -1                                       Unknown                NaN   
1         1                                Private flight                NaN   
2         2                                   135 Airways                NaN   
3         3                                 1Time Airline                NaN   
4         4  2 Sqn No 1 Elementary Flying Training School                NaN   
...     ...                                           ...                ...   
6157  21248                                   GX Airlines                NaN   
6158  21251                        Lynx Aviation (L3/SSX)                NaN   
6159  21268                               Jetgo Australia                NaN   
6160  21270                                  Air Carnival                NaN   
6161  21317                                 Svyaz Rossiya  Russian Commuter    

     IATA ICAO         CALLSIGN        

In [7]:
routes.head()

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
0,2B,410.0,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410.0,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410.0,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410.0,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410.0,CEK,2968.0,OVB,4078.0,NaN,0,CR2


In [8]:
print(routes)

      AIRLINE  AIRLINE_ID SOURCE  SOURCE_ID DEST  DEST_ID CODESHARE  STOPS  \
0          2B       410.0    AER     2965.0  KZN   2990.0       NaN      0   
1          2B       410.0    ASF     2966.0  KZN   2990.0       NaN      0   
2          2B       410.0    ASF     2966.0  MRV   2962.0       NaN      0   
3          2B       410.0    CEK     2968.0  KZN   2990.0       NaN      0   
4          2B       410.0    CEK     2968.0  OVB   4078.0       NaN      0   
...       ...         ...    ...        ...  ...      ...       ...    ...   
67658      ZL      4178.0    WYA     6334.0  ADL   3341.0       NaN      0   
67659      ZM     19016.0    DME     4029.0  FRU   2912.0       NaN      0   
67660      ZM     19016.0    FRU     2912.0  DME   4029.0       NaN      0   
67661      ZM     19016.0    FRU     2912.0  OSS   2913.0       NaN      0   
67662      ZM     19016.0    OSS     2913.0  FRU   2912.0       NaN      0   

      EQUIPMENT  
0           CR2  
1           CR2  
2        

In [9]:
planes.head()

,NAME,IATA,ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [10]:
countries.head()

,NAME,ISO,DAFIF
0,"Bonaire, Saint Eustatius and Saba",BQ,NaN
1,Aruba,AW,AA
2,Antigua and Barbuda,AG,AC
3,United Arab Emirates,AE,AE
4,Afghanistan,AF,AF


### Selecting data

We might want to work with a subset of data.  If so, we can select data just like we did in the previous lesson. 

In [11]:
# we can select all airports in one of the least visited countries on earth
airports[airports['COUNTRY']=="Tuvalu"]

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
3835,4077,Funafuti International Airport,Funafuti,Tuvalu,FUN,NGFU,-8.525,179.195999,9,12.0,U,Pacific/Funafuti,airport,OurAirports


In [12]:
#RUSSIA 
airports[airports['COUNTRY']=="Russia"]

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
2771,2923,Yakutsk Airport,Yakutsk,Russia,YKS,UEEE,62.093300,129.770996,325,9.0,N,Asia/Yakutsk,airport,OurAirports
2772,2925,Mirny Airport,Mirnyj,Russia,MJZ,UERR,62.534698,114.039001,1156,9.0,N,Asia/Yakutsk,airport,OurAirports
2773,2926,Ignatyevo Airport,Blagoveschensk,Russia,BQS,UHBB,50.425400,127.412003,638,9.0,N,Asia/Yakutsk,airport,OurAirports
2774,2927,Khabarovsk-Novy Airport,Khabarovsk,Russia,KHV,UHHH,48.528000,135.188004,244,10.0,N,Asia/Vladivostok,airport,OurAirports
2775,2929,Provideniya Bay Airport,Provideniya Bay,Russia,PVS,UHMD,64.378098,-173.242996,72,12.0,N,Asia/Anadyr,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7132,12006,Severomorsk-3 Naval Air Base,Severomorsk,Russia,NaN,XLMV,68.866669,33.716667,564,NaN,NaN,NaN,airport,OurAirports
7133,12007,Fedotovo Naval Air Base,Kipelovo,Russia,NaN,XLWF,59.188301,39.123299,574,NaN,NaN,NaN,airport,OurAirports
7134,12008,Lebyazhye Air Base,Kamyshin,Russia,NaN,XRWL,50.201698,45.208302,381,NaN,NaN,NaN,airport,OurAirports
7135,12009,Smolensk North Airport,Smolensk,Russia,NaN,XUBS,54.824000,32.025000,820,NaN,NaN,NaN,airport,OurAirports


In [12]:
# we can select all routes to that airport
routes[routes['DEST']=='FUN']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5


In [13]:
# and routes from that airport
routes[routes['SOURCE']=='FUN']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [17]:
# where are these flights going to/from? 


#####  Your turn: 

Select just the routes that go to or from Lexington Bluegrass Airport, and store them in their own dataframe.  

The airport code is LEX.  You should have a much smaller dataframe.  How many inbound routes and how many outbound routes are there? 

In [14]:
#New DataFrame 
#DataFrame destination TO LEX
df1 = routes[routes['DEST']=='LEX']

In [15]:
df1

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


In [21]:
#New DataFrame 
#DataFrame destination FROM LEX
df2 = routes[routes['SOURCE']=='LEX']

In [22]:
df2

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
21095,DL,2009.0,LEX,4017.0,ATL,3682.0,NaN,0,M88 717
21096,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ
21097,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9
21098,DL,2009.0,LEX,4017.0,LGA,3697.0,NaN,0,ERJ
21099,DL,2009.0,LEX,4017.0,MSP,3858.0,Y,0,CRJ


In [23]:
#COMBINED TO/FROM LEX BY USING APPEND, SHORTCUT FOR CONCATE METHOD 
result = df1.append(df2)
display(result)

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


In [24]:
#New DataFrame for to/from LEX
dfNew = result
dfNew

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


In [25]:
# By using Concatentating method
to_lexington = routes[routes['DEST']=='LEX']
from_lexington = routes[routes['SOURCE']=='LEX']

In [26]:
pd.concat([to_lexington, from_lexington])

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


### Concatentating data

To concatenate is to link things together in a series or chain.  It is similar to, but more powerful than appending data, because it can work on either rows or columns, is faster, and will perform set logic (union or intersection).  

In [27]:
# let's go back and contatenate the routes both to and from Tuvalu
to_tuvalu = routes[routes['DEST']=='FUN']
from_tuvalu = routes[routes['SOURCE']=='FUN']


In [28]:
# the basic function, given a list of dataframes to concatenate
# note that we call concatenate as a "static" method, rather than a method 
# associated with a specific object. 

pd.concat([to_tuvalu, from_tuvalu])

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [29]:
# if we want to use the dataframe later, we need to assign it to a variable name

tuvalu_routes = pd.concat([to_tuvalu, from_tuvalu])

Note to see more about this method, we can examine the pandas docs:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [30]:
# note that one of the arguments is axis.  
# axis=0 indicates that the rows should be concatenated, 
# while axis=1 indicates to concatenate the columns 

pd.concat([to_tuvalu, from_tuvalu], axis=1)

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0.0,AT5
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Your turn:

Can you use the concat() function to create one dataframe with flights both into and out of LEX?

In [31]:
# By using Concatentating method
to_lexington = routes[routes['DEST']=='LEX']
from_lexington = routes[routes['SOURCE']=='LEX']

In [32]:
pd.concat([to_lexington, from_lexington])

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


In [33]:
#New DataFrame for concate method 
dfNEW2 = pd.concat([to_lexington, from_lexington])
dfNEW2

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


### Merging data

Sometimes we want to merge, or join, the data instead of concatenating it.  For example, maybe we hate flying on small planes, so want to know what kind of aircraft we would need to fly on to visit Tuvalu.  This data comes from a different table. 

In [34]:
# we see that the Tuvalu routes operate with AT5 equipment
tuvalu_routes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [35]:
# but what is an AT5? Let's look in the planes table. 
planes.head()

,NAME,IATA,ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [36]:
# in this case, we are lucky, and it is in the first few rows.  But to do this systemactically, we
# would want to join the tables.  To do this, we use the merge() function in pandas.  

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


Let's stop and look at what is going on in the above command. 

It is important to understand the type of merge, as specified by the how option.  Here is what the pandas docs say:


![alt text](img/how.png)


![alt text](img/how2.png)

So it looks like the above merge defaulted to an inner join


![alt text](img/inner.png)

In [37]:
# we can check this by explicitly setting the option and confirming that we get the same result  
# next, let's check what happens if we choose the other options

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='inner')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [38]:
# outer join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='outer')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Nord) 262,ND2,N262
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale SN.601 Corvette,NDC,S601
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-134,TU3,T134
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-154,TU5,T154
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-204,T20,T204
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yakovlev Yak-40,YK4,YK40


In [39]:
# left join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='left')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [40]:
# right join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='right')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Nord) 262,ND2,N262
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale SN.601 Corvette,NDC,S601
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-144,NaN,T144
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-154,TU5,T154
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-204,T20,T204
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yakovlev Yak-40,YK4,YK40


I find that I most often want a left join, as described here: 

![alt text](img/left.png)

It may be worth examining some of the other options used in merge().  We can refer to the pandas documentation to learn more. 

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging

#### Your turn

Now let's look at which airlines operate in and out of Lexington.  How many routes does each airline have?  

First, let's spend some time planning together.  Then individually take some time to implement the code we planned. 

In [41]:
#Airlines that operate in and out of LEX
dfNEW2.AIRLINE.unique()

array(['9E', 'AA', 'AF', 'DL', 'G4', 'KL', 'UA', 'US'], dtype=object)

In [42]:
#Routs that each airline has
dfNEW2['AIRLINE'].value_counts()

DL    10
G4     8
AA     6
US     6
UA     4
9E     2
AF     2
KL     2
Name: AIRLINE, dtype: int64

#### It looks like there are some international airlines with Lexington routes.  

To look at how many routes they have, create a new column in your dataframe called 'International', which is set to True for an overseas airline and False for a domestic airline.  Calculate the percent of routes with an overseas airline.  

You should have the tools you need to answer this question using what we learned in last week's lesson.  Again, let's start by planning together, then implement the code on your own.  

In [43]:
# United States Airlines 
df3 = airlines[airlines['COUNTRY']=='United States']
df3

,ID,NAME,ALIAS,IATA,ICAO,CALLSIGN,COUNTRY,ACTIVE
2,2,135 Airways,NaN,NaN,GNL,GENERAL,United States,N
9,9,3D Aviation,NaN,NaN,SEC,SECUREX,United States,N
10,10,40-Mile Air,NaN,Q5,MLA,MILE-AIR,United States,Y
22,22,Aloha Airlines,NaN,AQ,AAH,ALOHA,United States,Y
23,23,Alaska Island Air,NaN,NaN,AAK,ALASKA ISLAND,United States,N
...,...,...,...,...,...,...,...,...
6141,20978,All America US,All America US,AG,SSA,NaN,United States,Y
6143,20991,Choice Airways,NaN,NaN,CSX,NaN,United States,Y
6146,20996,Dynamic Airways,NaN,NaN,DYA,NaN,United States,Y
6152,21124,ViaAir,Charter Air Transport,NaN,SRY,Stingray,United States,Y


In [44]:
#Unique values 
df3.IATA.unique()

array([nan, 'Q5', 'AQ', 'AA', 'G4', 'GB', '4G', '7A', 'KO', '2Q', '5A',
       'M6', 'HP', 'ZW', 'ZV', '6V', 'JW', 'EV', '8C', 'MQ', '5F', 'ZA',
       'ER', 'QH', '5Y', 'MC', 'FL', '8Q', 'B4', 'CH', '8E', 'BN', 'GQ',
       'E9', 'R9', '9K', 'PT', 'GG', 'WE', 'OP', 'MG', 'RP', 'C8', 'DQ',
       '9L', 'OH', 'C5', 'CP', 'CO', 'CS', 'DL', 'DH', 'W9', 'EM', 'E0',
       'EZ', 'OW', 'EO', 'XE', 'FX', 'PA', 'RF', 'HK', 'FP', 'F9', '2F',
       '1G', 'GR', 'G7', 'GS', 'ZK', 'H6', 'HB', 'HA', 'QX', 'II', '1U',
       'I9', 'IS', '2S', 'QJ', 'B6', 'K4', 'M5', 'KP', 'JF', 'L2', 'MY',
       'YV', 'XJ', 'GL', 'JI', 'YX', 'DV', 'N7', '1N', '1I', 'EJ', 'NC',
       'NW', 'OY', 'OZ', 'Y5', 'PS', 'LW', 'KS', 'PI', '9E', 'PO', 'BK',
       'FE', '1D', '3C', 'QQ', 'RW', 'E2', 'RD', 'BB', 'SY', 'N5', 'WN',
       'AL', 'SO', '2G', '1S', '1K', 'NK', 'SX', 'OO', '8D', 'S5', 'TI',
       'FF', 'T9', 'TW', 'AX', '1L', 'U5', 'UA', 'U2', 'U7', '5X', 'US',
       'UH', 'VX', 'WA', 'CN', 'K5', 'WO', '1P', '8V

In [45]:
#By doing this, we find that AF and KL are international airlines. I was not sure how to do it automaticly
df3[df3['IATA'] == 'KL']

,ID,NAME,ALIAS,IATA,ICAO,CALLSIGN,COUNTRY,ACTIVE


In [50]:
# I am not sure how to know which ine is international and which one is not 
dfNEW2['International'] = 'True'
dfNEW2

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,International
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,True
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,True
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,True
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,True
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,True
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717,True
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ,True
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717,True
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ,True
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ,True


In [49]:
# International airlines 
df4 = dfNEW2
df4

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,International
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,True
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,True
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,True
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,True
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,True
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717,True
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ,True
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717,True
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ,True
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ,True


In [51]:
# This identifies if the airline is international or not. I am not sure how to suttisfy both of the airlines. 
df4.loc[df4['AIRLINE'] == 'AF', 'International'] = 'True'  
df4.loc[df4['AIRLINE'] != 'AF', 'International'] = 'False'  

print (df4)
    

      AIRLINE  AIRLINE_ID SOURCE  SOURCE_ID DEST  DEST_ID CODESHARE  STOPS  \
3569       9E      3976.0    ATL     3682.0  LEX   4017.0       NaN      0   
4953       AA        24.0    CLT     3876.0  LEX   4017.0       NaN      0   
5247       AA        24.0    DFW     3670.0  LEX   4017.0         Y      0   
6283       AA        24.0    ORD     3830.0  LEX   4017.0         Y      0   
9097       AF       137.0    ATL     3682.0  LEX   4017.0         Y      0   
20164      DL      2009.0    ATL     3682.0  LEX   4017.0       NaN      0   
20534      DL      2009.0    DCA     3520.0  LEX   4017.0         Y      0   
20638      DL      2009.0    DTW     3645.0  LEX   4017.0       NaN      0   
21131      DL      2009.0    LGA     3697.0  LEX   4017.0         Y      0   
21402      DL      2009.0    MSP     3858.0  LEX   4017.0         Y      0   
28951      G4        35.0    FLL     3533.0  LEX   4017.0       NaN      0   
29090      G4        35.0    PGD     7056.0  LEX   4017.0       

#### Codeshares

It looks like a bunch of these routes are codeshares.  That means they are marketed by this airline, but operated by a different airline.  See the note in the data documentation on openflights.org/data.  The implication of this is that there are duplicates.

Can you figure out which ones are duplicates?  Can you then create a dataframe with only the unique routes?  How many unique inbound and outbound routes are there? 

Remember, someone has to operate the flight, so if all the routes to/from a particular airport are listed as codeshares, then something is funny...

It is also possible that more than one airline actually operates a route between the same two airports. (Having this sort of competition generally means that you will get better fares as a traveler.)  It may not be obvious what is actually in the data set, so dig or do external research as needed.  

It is this sort of messiness that will consume much of your effort in research and data science.  It is very important to "become one with your data". 

Figuring this out will be the first part of your homework.  

### Review Questions: 

1. What is a key in relational data? 

2. Explain the difference between a left join and an outer join.  

3. Explain the difference between pd.concat() and pd.merge()

4. Identify two possible places to look for help if you get stuck on the homework. 

In [ ]:
#Key in relational data is 

In [ ]:
# The left join uses the key join to combine two dataframes and it will 
#return all of rows from the left even if they do not match. 
#There are two types of outer join: Left ad Right. Right is the same as 
#Left only it will return all the rows from the right 

In [ ]:
# pd.concat() is combining two dataframes into one, placing the one after another. 
# pd.merge() is combines two dataframes based on similar columns. 

In [ ]:
# Google


### Homework 1 - Code shares

Dig into the data as described above, and do your best to identify which routes are code shares, and how many unique routes operate into/out of Lexington.

In [52]:
#Unique routs 
dfNEW2.DEST.unique()

array(['LEX', 'ATL', 'CLT', 'DFW', 'ORD', 'DCA', 'DTW', 'LGA', 'MSP',
       'FLL', 'PGD', 'PIE', 'SFB', 'IAH'], dtype=object)

In [53]:
#Numbers of unique routs 
dfNEW2['DEST'].value_counts()

LEX    20
ATL     4
ORD     3
CLT     2
DFW     2
DCA     1
DTW     1
LGA     1
MSP     1
FLL     1
PGD     1
PIE     1
SFB     1
IAH     1
Name: DEST, dtype: int64

In [54]:
#Routes that are codeshare 
dfNEW2[dfNEW2['CODESHARE'] == 'Y']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,International
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,False
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,False
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,True
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ,False
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ,False
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ,False
36396,KL,3090.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ,False
56775,UA,5209.0,IAH,3550.0,LEX,4017.0,Y,0,ERJ,False
57322,UA,5209.0,ORD,3830.0,LEX,4017.0,Y,0,ERJ CRJ,False
59924,US,5265.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,False


### Homework 2 - The Kevin Bacon Game

Of course, it is possible to fly beyond those few cities from Lexington--you just have to transfer to get there.  Your task is to:

1. Identify which airports you can get to from Lexington with only one transfer.  
2. Identify which airports you can get to from Lexington with two transfers.

You have the tools to do this using the merge functionality presented today, but you will need to be smart about what you are merging.  

Next class, we will use these data to make interactive maps of these flight routes, similar to what you see in the back of the airline magazines.  


#### Bonus: 

This is the air travel version of the Kevin Bacon game (https://oracleofbacon.org/). What is the number N, such that you can reach every airport in the world with N or fewer transfers?

#### Extra Bonus: 

Use this very important piece of knowledge to impress your friends at parties!

In [55]:
# SOurce Lexington 
to_lexington = routes[routes['SOURCE']=='LEX']
to_lexington

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
21095,DL,2009.0,LEX,4017.0,ATL,3682.0,NaN,0,M88 717
21096,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ
21097,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9
21098,DL,2009.0,LEX,4017.0,LGA,3697.0,NaN,0,ERJ
21099,DL,2009.0,LEX,4017.0,MSP,3858.0,Y,0,CRJ


In [56]:
# Identifying all the airports that you can get to from lex
to_lexington.DEST.unique()

array(['ATL', 'CLT', 'DFW', 'ORD', 'DCA', 'DTW', 'LGA', 'MSP', 'FLL',
       'PGD', 'PIE', 'SFB', 'IAH'], dtype=object)

In [57]:
from_ATL = routes[routes['SOURCE']=='ATL']
from_CLT = routes[routes['SOURCE']=='CLT']
from_DFW = routes[routes['SOURCE']=='DFW']
from_ORD = routes[routes['SOURCE']=='ORD']
from_DCA = routes[routes['SOURCE']=='DCA']
from_DTW = routes[routes['SOURCE']=='DTW']
from_DTW = routes[routes['SOURCE']=='LGA']
from_MSP = routes[routes['SOURCE']=='MSP']
from_FLL = routes[routes['SOURCE']=='FLL']
from_PGD = routes[routes['SOURCE']=='PGD']
from_PIE = routes[routes['SOURCE']=='PIE']
from_SFB = routes[routes['SOURCE']=='SFB']
from_IAH = routes[routes['SOURCE']=='IAH']


In [75]:
# With just one transfer 
pd.concat([from_ATL,from_CLT,from_DFW,from_ORD,from_DCA,from_DTW,from_MSP,from_FLL,from_PGD,from_PIE,from_SFB, from_IAH])

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
443,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3
444,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3
445,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3
446,3M,20710.0,ATL,3682.0,MSL,5756.0,NaN,0,SF3
447,3M,20710.0,ATL,3682.0,PIB,5759.0,NaN,0,SF3
...,...,...,...,...,...,...,...,...,...
59370,US,5265.0,IAH,3550.0,PHX,3462.0,NaN,0,319 320
59371,US,5265.0,IAH,3550.0,SAL,1892.0,NaN,0,E90
61079,VA,5360.0,IAH,3550.0,ATL,3682.0,Y,0,M88 717 319
61274,VB,10646.0,IAH,3550.0,MTY,1825.0,NaN,0,733


In [84]:
one_trans = pd.concat([from_ATL,from_CLT,from_DFW,from_ORD,from_DCA,from_DTW,from_MSP,from_FLL,from_PGD,from_PIE,from_SFB, from_IAH])
one_trans

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
443,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3
444,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3
445,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3
446,3M,20710.0,ATL,3682.0,MSL,5756.0,NaN,0,SF3
447,3M,20710.0,ATL,3682.0,PIB,5759.0,NaN,0,SF3
...,...,...,...,...,...,...,...,...,...
59370,US,5265.0,IAH,3550.0,PHX,3462.0,NaN,0,319 320
59371,US,5265.0,IAH,3550.0,SAL,1892.0,NaN,0,E90
61079,VA,5360.0,IAH,3550.0,ATL,3682.0,Y,0,M88 717 319
61274,VB,10646.0,IAH,3550.0,MTY,1825.0,NaN,0,733


In [85]:
# All the unique destinations for 1 transfer 
one_trans = one_trans.drop_duplicates(subset=['DEST'])
one_trans

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
443,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3
444,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3
445,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3
446,3M,20710.0,ATL,3682.0,MSL,5756.0,NaN,0,SF3
447,3M,20710.0,ATL,3682.0,PIB,5759.0,NaN,0,SF3
...,...,...,...,...,...,...,...,...,...
56846,UA,5209.0,IAH,3550.0,TAM,1848.0,Y,0,CR7 ERJ
56855,UA,5209.0,IAH,3550.0,VER,1854.0,Y,0,ERJ
56857,UA,5209.0,IAH,3550.0,VSA,1853.0,Y,0,CR7
56864,UA,5209.0,IAH,3550.0,ZIH,1856.0,Y,0,ERJ


In [86]:
# This is where you can get just with one transfer
one_trans.DEST.unique()

array(['LWB', 'MCN', 'MEI', 'MSL', 'PIB', 'TUP', 'AZO', 'CHA', 'CID',
       'CRW', 'CVG', 'EVV', 'FWA', 'GSO', 'GSP', 'LAN', 'LEX', 'MBS',
       'MSP', 'ROA', 'SYR', 'TYS', 'XNA', 'CLT', 'DFW', 'LHR', 'MIA',
       'ORD', 'PHL', 'PHX', 'DEN', 'YYZ', 'ABE', 'ABQ', 'ABY', 'AEX',
       'AGS', 'ALB', 'ATW', 'AUS', 'AVL', 'AVP', 'BDL', 'BHM', 'BMI',
       'BNA', 'BOS', 'BQK', 'BRU', 'BTR', 'BTV', 'BUF', 'BWI', 'CAE',
       'CAK', 'CDG', 'CHO', 'CHS', 'CLE', 'CMH', 'COS', 'CSG', 'CUN',
       'CZM', 'DAB', 'DAL', 'DAY', 'DCA', 'DHN', 'DSM', 'DTW', 'DUB',
       'DUS', 'ECP', 'ELP', 'EWN', 'EWR', 'EYW', 'FAY', 'FCO', 'FLL',
       'FNT', 'FRA', 'FSM', 'GDL', 'GNV', 'GPT', 'GRB', 'GRK', 'GRR',
       'GTR', 'HPN', 'HSV', 'IAD', 'IAH', 'ICT', 'ILM', 'IND', 'JAN',
       'JAX', 'JFK', 'LAS', 'LAX', 'LFT', 'LGA', 'LIT', 'MAD', 'MAN',
       'MCI', 'MCO', 'MDT', 'MDW', 'MEM', 'MEX', 'MGM', 'MHT', 'MKE',
       'MLB', 'MLI', 'MLU', 'MOB', 'MSN', 'MSY', 'MTY', 'MUC', 'MYR',
       'NAS', 'OAJ',

In [87]:
# 2. Where can you get with two transfers. Now, I cannot do it manualy as there are too many airports to identify.

two_trans = pd.merge(one_trans, routes, left_on='DEST', right_on='SOURCE', how='left')
two_trans

,AIRLINE_x,AIRLINE_ID_x,SOURCE_x,SOURCE_ID_x,DEST_x,DEST_ID_x,CODESHARE_x,STOPS_x,EQUIPMENT_x,AIRLINE_y,AIRLINE_ID_y,SOURCE_y,SOURCE_ID_y,DEST_y,DEST_ID_y,CODESHARE_y,STOPS_y,EQUIPMENT_y
0,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0.0,SF3
1,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3,UA,5209.0,LWB,6958.0,IAD,3714.0,Y,0.0,SF3
2,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3,3M,20710.0,MCN,3754.0,ATL,3682.0,NaN,0.0,SF3
3,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3,3M,20710.0,MCN,3754.0,MCO,3878.0,NaN,0.0,SF3
4,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3,3M,20710.0,MEI,4335.0,ATL,3682.0,NaN,0.0,SF3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26153,UA,5209.0,IAH,3550.0,ZLO,1858.0,Y,0,ERJ,AM,321.0,ZLO,1858.0,MEX,1824.0,Y,0.0,E75 ERJ
26154,UA,5209.0,IAH,3550.0,ZLO,1858.0,Y,0,ERJ,AS,439.0,ZLO,1858.0,LAX,3484.0,NaN,0.0,73H
26155,UA,5209.0,IAH,3550.0,ZLO,1858.0,Y,0,ERJ,UA,5209.0,ZLO,1858.0,IAH,3550.0,Y,0.0,ERJ
26156,UA,5209.0,IAH,3550.0,ZLO,1858.0,Y,0,ERJ,VW,1290.0,ZLO,1858.0,MEX,1824.0,NaN,0.0,ATR


In [88]:
two_trans = two_trans.drop_duplicates(subset=['DEST_y'])
two_trans

,AIRLINE_x,AIRLINE_ID_x,SOURCE_x,SOURCE_ID_x,DEST_x,DEST_ID_x,CODESHARE_x,STOPS_x,EQUIPMENT_x,AIRLINE_y,AIRLINE_ID_y,SOURCE_y,SOURCE_ID_y,DEST_y,DEST_ID_y,CODESHARE_y,STOPS_y,EQUIPMENT_y
0,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0.0,SF3
1,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3,UA,5209.0,LWB,6958.0,IAD,3714.0,Y,0.0,SF3
3,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3,3M,20710.0,MCN,3754.0,MCO,3878.0,NaN,0.0,SF3
8,3M,20710.0,ATL,3682.0,TUP,5773.0,NaN,0,SF3,3M,20710.0,TUP,5773.0,GLH,6130.0,NaN,0.0,SF3
10,9E,3976.0,ATL,3682.0,AZO,4039.0,NaN,0,CRJ,AA,24.0,AZO,4039.0,ORD,3830.0,Y,0.0,ER4 ERD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25999,SQ,4435.0,IAH,3550.0,DME,4029.0,NaN,0,77W,UN,5067.0,DME,4029.0,PKC,2932.0,NaN,0.0,773
26024,SQ,4435.0,IAH,3550.0,DME,4029.0,NaN,0,77W,VN,5309.0,DME,4029.0,CXR,6189.0,NaN,0.0,332 330
26034,SQ,4435.0,IAH,3550.0,DME,4029.0,NaN,0,77W,YC,9828.0,DME,4029.0,NOJ,4365.0,NaN,0.0,735
26037,SQ,4435.0,IAH,3550.0,DME,4029.0,NaN,0,77W,YC,9828.0,DME,4029.0,SLY,6137.0,NaN,0.0,735 320
